In [3]:
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

from IPython.display import clear_output

In [4]:
dir_img = '/home/vlad/dataset_new/train_data/imgs/'
dir_mask = '/home/vlad/dataset_new/train_data/masks/'

dir_checkpoint = 'ckpts_dir_new/mid_ckpts/'

try:
# Create target Directory
    os.mkdir(dir_checkpoint)
    print("Directory " , dir_checkpoint ,  " Created ") 
except FileExistsError:
    print("Directory " , dir_checkpoint ,  " already exists")

Directory  ckpts_dir_new/mid_ckpts/  already exists


In [5]:
def train_net(net,
              device,
              epochs=5,
              batch_size=1,
              lr=0.001,
              val_percent=0.1,
              save_cp=True,
              img_scale=0.5,
              epoch_bias = 0):
    logging.basicConfig(filename='/home/vlad/unet4/ckpts_dir_new/logs.log', level=logging.INFO)
    dataset = BasicDataset(dir_img, dir_mask, img_scale)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 2 else 'max', patience=2)
    if net.n_classes > 2:
        print("Using CrossEntropyLoss")
        criterion = nn.CrossEntropyLoss()
    else:
        print("Using BCEWithLogitsLoss")
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                mask_type = torch.float32 if net.n_classes <= 2 else torch.long
                true_masks = true_masks.to(device=device, dtype=mask_type)

                masks_pred = net(imgs)
                
                loss = criterion(masks_pred, true_masks)
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                if global_step % (len(dataset) // (10 * batch_size)) == 0:
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)

                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                    if net.n_classes > 2:
                        logging.info('Validation cross entropy: {}'.format(val_score))
                        writer.add_scalar('Loss/test', val_score, global_step)
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val_score))
                        writer.add_scalar('Dice/test', val_score, global_step)

                    writer.add_images('images', imgs, global_step)
                    if net.n_classes == 1:
                        writer.add_images('masks/true', true_masks, global_step)
                        writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            if (epoch + epoch_bias + 1) % 50 == 0:
                torch.save(net.state_dict(),
                           dir_checkpoint + f'CP_epoch{epoch + epoch_bias + 1}.pth')
                logging.info(f'Checkpoint {epoch + epoch_bias + 1} saved !')

    writer.close()

In [6]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [7]:
net = UNet(n_channels=1, n_classes=2, bilinear=True)
logging.info(f'Network:\n'
             f'\t{net.n_channels} input channels\n'
             f'\t{net.n_classes} output channels (classes)\n'
             f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')

INFO: Network:
	1 input channels
	2 output channels (classes)
	Bilinear upscaling


In [8]:
n_classes = 2 
n_channels = 1 

In [9]:
%%capture cap --no-stderr

epochs = 100 
batchsize = 16

load = False
epoch_bias = 0

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

INFO: Creating dataset with 1288 examples
I0909 16:46:08.855720 140398826483840 <ipython-input-5-ff186211403b>:30] Starting training:
        Epochs:          100
        Batch size:      16
        Learning rate:   0.001
        Training size:   1160
        Validation size: 128
        Checkpoints:     True
        Device:          cuda
        Images scaling:  0.5
    
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.25batch/s]
                                                                  I0909 16:46:17.684962 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.36batch/s]
                                                                  I0909 16:46:26.102658 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.35batch/s]
                                                                  I0909 16:46:34.552503 1403

                                                                  I0909 16:47:25.105591 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.29batch/s]
                                                                  I0909 16:47:33.654936 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.40batch/s]
                                                                  I0909 16:47:42.027500 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 16:47:50.476880 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.21batch/s]
                                                                  I0909 16:48:49.705541 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.29batch/s]
                                                                  I0909 16:48:58.215776 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.36batch/s]
                                                                  I0909 16:49:06.689218 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 16:49:15.123869 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
 

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.32batch/s]
                                                                  I0909 16:50:06.129248 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 16:50:14.657116 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.35batch/s]
                                                                  I0909 16:50:23.096211 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0909 16:50:31.622335 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.29batch/s]
 

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.28batch/s]
                                                                  I0909 16:51:22.740041 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.33batch/s]
                                                                  I0909 16:51:31.221155 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.35batch/s]
                                                                  I0909 16:51:39.696097 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.35batch/s]
                                                                  I0909 16:51:48.223410 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.36batch/s]
 

...

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.32batch/s]
                                                                  I0909 18:50:03.128964 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.27batch/s]
                                                                  I0909 18:50:11.625066 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.35batch/s]
                                                                  I0909 18:50:20.144541 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.27batch/s]
                                                                  I0909 18:50:28.375492 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.36batch/s]


Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.33batch/s]
                                                                  I0909 18:51:19.814081 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.29batch/s]
                                                                  I0909 18:51:28.304364 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0909 18:51:36.728206 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 18:51:45.280176 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
 

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.34batch/s]
                                                                  I0909 18:52:36.424972 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0909 18:52:45.032669 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.32batch/s]
                                                                  I0909 18:52:53.550979 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 18:53:02.018511 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
 

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0909 18:53:52.991375 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.34batch/s]
                                                                  I0909 18:54:01.467289 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.28batch/s]
                                                                  I0909 18:54:09.967990 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.29batch/s]
                                                                  I0909 18:54:18.475681 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
 

Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.25batch/s]
                                                                  I0909 18:55:18.104331 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.31batch/s]
                                                                  I0909 18:55:26.659769 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 8/8 [00:01<00:00,  5.30batch/s]
                                                                  I0909 18:55:35.127769 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Epoch 100/100: 100%|██████████| 1160/1160 [01:17<00:00, 14.66img/s, loss (batch)=1.23e-5]
I0909 18:55:37.082769 140398826483840 <ipython-input-5-ff186211403b>:105] Checkpoint 100 saved !


In [10]:
with open('ckpts_dir_new/output_init.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap --no-stderr

epochs = 100 
batchsize = 8

load = dir_checkpoint+'CP_epoch100.pth'
epoch_bias = 100

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

I0909 18:55:37.124861 140398826483840 <ipython-input-11-34d56e1c2201>:11] Model loaded from ckpts_dir_new/mid_ckpts/CP_epoch100.pth
I0909 18:55:37.128268 140398826483840 dataset.py:24] Creating dataset with 1288 examples
I0909 18:55:37.129369 140398826483840 <ipython-input-5-ff186211403b>:30] Starting training:
        Epochs:          100
        Batch size:      8
        Learning rate:   0.001
        Training size:   1160
        Validation size: 128
        Checkpoints:     True
        Device:          cuda
        Images scaling:  0.5
    
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.47batch/s]
                                                                    I0909 18:55:45.872941 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.45batch/s]
                                                                    I0909 18:55:54.444877 140398826483840 <ipython-input-5-ff186211403b>:88] 

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.50batch/s]
                                                                    I0909 18:56:54.590291 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.20batch/s]
                                                                    I0909 18:57:03.275670 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.36batch/s]
                                                                    I0909 18:57:11.896934 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.35batch/s]
                                                                    I0909 18:57:20.500560 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.52batch/s]
                                                                    I0909 18:58:12.083459 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.18batch/s]
                                                                    I0909 18:58:20.900398 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.36batch/s]
                                                                    I0909 18:58:29.556002 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.39batch/s]
                                                                    I0909 18:58:38.161414 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.46batch/s]
                                                                    I0909 18:59:29.829513 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.57batch/s]
                                                                    I0909 18:59:38.498327 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round: 100%|██████████| 16/16 [00:01<00:00, 10.04batch/s]
                                                                    I0909 18:59:47.132686 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.49batch/s]
                                                                    I0909 18:59:55.748083 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.40batch/s]
                                                                    I0909 19:00:47.384410 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.48batch/s]
                                                                    I0909 19:00:56.093596 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.48batch/s]
                                                                    I0909 19:01:04.693650 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.44batch/s]
                                                                    I0909 19:01:13.312786 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

...

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.40batch/s]
                                                                    I0909 19:26:39.098502 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.49batch/s]
                                                                    I0909 19:26:47.664586 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.48batch/s]
                                                                    I0909 19:26:56.368195 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.52batch/s]
                                                                    I0909 19:27:04.944604 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  8.83batch/s]
                                                                    I0909 19:27:56.550563 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.37batch/s]
                                                                    I0909 19:28:05.254682 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.37batch/s]
                                                                    I0909 19:28:14.034941 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.53batch/s]
                                                                    I0909 19:28:22.651746 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.33batch/s]
                                                                    I0909 19:29:14.240676 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.49batch/s]
                                                                    I0909 19:29:22.872015 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.31batch/s]
                                                                    I0909 19:29:31.601533 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.30batch/s]
                                                                    I0909 19:29:40.189527 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round: 100%|██████████| 16/16 [00:01<00:00, 10.05batch/s]
                                                                    I0909 19:30:31.869435 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.59batch/s]
                                                                    I0909 19:30:40.456816 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.44batch/s]
                                                                    I0909 19:30:49.188869 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.46batch/s]
                                                                    I0909 19:30:57.824045 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.35batch/s]
                                                                    I0909 19:31:49.455196 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.37batch/s]
                                                                    I0909 19:31:58.075014 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.47batch/s]
                                                                    I0909 19:32:06.773230 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:00,  9.30batch/s]
                                                                    I0909 19:32:15.401760 140398826483840 <ipython-input-5-ff186211403b>:88] Validation Dice Coeff: 1.0
Validation round:  94%|█████████▍| 15/16 [00:01<00:0

In [ ]:
with open('ckpts_dir_new/output_CP_epoch100.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap --no-stderr

epochs = 50 
batchsize = 4

load = dir_checkpoint+'CP_epoch200.pth'
epoch_bias = 200

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

In [ ]:
with open('ckpts_dir_new/output_CP_epoch200.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap --no-stderr

epochs = 50 
batchsize = 2

load = dir_checkpoint+'CP_epoch250.pth'
epoch_bias = 250

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

In [ ]:
with open('ckpts_dir_new/output_CP_epoch250.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap --no-stderr

epochs = 100 
batchsize = 1

load = dir_checkpoint+'CP_epoch300.pth'
epoch_bias = 300

if load:
    net.load_state_dict(
        torch.load(load, map_location=device)
    )
    logging.info(f'Model loaded from {load}')

net.to(device=device)

# clear_output()

try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batchsize,
              device=device,
              epoch_bias=epoch_bias)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

In [ ]:
with open('ckpts_dir_new/output_CP_epoch300.txt', 'w') as f:
    f.write(cap.stdout)